In [2]:
#! pip install --upgrade pip
#! pip install pandas

import os
import pandas as pd

database_name = "basbunar2u_denverCrimes"
csv_dir_path = "./csv_files/"
sql_dir_path = "./sql_files/"

csv_files_names = os.listdir(csv_dir_path)
csv_files_names = [name for name in csv_files_names if name.endswith(".csv")]
csv_files_names


for file in csv_files_names:
    
    df = pd.read_csv(csv_dir_path+str(file))
    table_name = file.split(".")[0]
    file_name = sql_dir_path+table_name+".sql"

    noms_des_colonnes = df.columns
    df.rename(columns=lambda x: x.lower(), inplace=True)
    df['incident_address'] = df['incident_address'].str.replace("'", "\\'")
    df['incident_address'] = df['incident_address'].str.replace('"', '\\"')
    df['first_occurrence_date'] = pd.to_datetime(df['first_occurrence_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
    df['first_occurrence_date'] = df['first_occurrence_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df['reported_date'] = pd.to_datetime(df['reported_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
    df['reported_date'] = df['reported_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df['last_occurrence_date'] = pd.to_datetime(df['last_occurrence_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
    df['last_occurrence_date'] = df['last_occurrence_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    def ma_fonction(valeur):
        if not pd.isna(valeur):
            return "'"+valeur+"'"
        else:
            return valeur 

    df['last_occurrence_date'] = df['last_occurrence_date'].apply(ma_fonction)
    df['incident_address'] = df['incident_address'].apply(ma_fonction)
    df['neighborhood_id'] = df['neighborhood_id'].apply(ma_fonction)
    df['district_id'] = df['district_id'].apply(ma_fonction)


    df['last_occurrence_date'] = df['last_occurrence_date'].fillna("NULL")
    df['incident_address'] = df['incident_address'].fillna("NULL")
    df['geo_x'] = df['geo_x'].fillna("NULL")
    df['geo_y'] = df['geo_y'].fillna("NULL")
    df['geo_lon'] = df['geo_lon'].fillna("NULL")
    df['geo_lat'] = df['geo_lat'].fillna("NULL")
    df['neighborhood_id'] = df['neighborhood_id'].fillna("NULL")
    df['district_id'] = df['district_id'].fillna("NULL")

    df['first_occurrence_date'] = pd.to_datetime(df['first_occurrence_date'])
    df['reported_date'] = pd.to_datetime(df['reported_date'])

    if not os.path.exists(sql_dir_path):
        os.makedirs(sql_dir_path)

    sql_file = open(file_name, 'w')

    create_table_query = f"""
    CREATE TABLE {database_name}.{table_name} (
        incident_id BIGINT NOT NULL,
        offense_id BIGINT NOT NULL, 
        offense_code INT NOT NULL, 
        offense_code_extension INT NOT NULL,
        offense_type_id VARCHAR(40) NOT NULL, 
        offense_category_id VARCHAR(30) NOT NULL,
        first_occurrence_date DATETIME NOT NULL,
        last_occurrence_date DATETIME,
        reported_date DATETIME NOT NULL,
        incident_address TEXT, 
        geo_x DOUBLE,
        geo_y DOUBLE, 
        geo_lon DOUBLE, 
        geo_lat DOUBLE, 
        district_id CHAR(1), 
        precinct_id INT NOT NULL,
        neighborhood_id VARCHAR(50), 
        is_crime INT NOT NULL, 
        is_traffic INT NOT NULL, 
        victim_count INT NOT NULL
    );
    """
    sql_file.write(create_table_query)

    for _, row in df.iterrows():

        insert_query = f"""
        INSERT INTO {table_name} (incident_id,offense_id,offense_code,offense_code_extension,offense_type_id,offense_category_id,first_occurrence_date,last_occurrence_date,reported_date,incident_address,geo_x,geo_y,geo_lon,geo_lat,district_id,precinct_id,neighborhood_id,is_crime,is_traffic,victim_count)

        VALUES ({row['incident_id']}, 
                '{row['offense_id']}', 
                {row['offense_code']},
                {row['offense_code_extension']},
                '{row['offense_type_id']}',
                '{row['offense_category_id']}',
                '{row['first_occurrence_date'].strftime('%Y-%m-%d %H:%M:%S')}',
                {row['last_occurrence_date']},
                '{row['reported_date'].strftime('%Y-%m-%d %H:%M:%S')}',
                {row['incident_address']},
                {row['geo_x']},
                {row['geo_y']},
                {row['geo_lon']},
                {row['geo_lat']},
                {row['district_id']},
                {row['precinct_id']},
                {row['neighborhood_id']},
                {row['is_crime']},
                {row['is_traffic']},
                {row['victim_count']}
                );
        """

        sql_file.write(insert_query)

    print(f"Commandes SQL enregistrées dans le fichier {file_name}")
    sql_file.close()


Commandes SQL enregistrées dans le fichier ./sql_files/crime2018.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2019.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2021.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2020.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2012.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2013.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2011.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2010.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2014.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2015.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2017.sql
Commandes SQL enregistrées dans le fichier ./sql_files/crime2016.sql
